<a href="https://colab.research.google.com/github/fenwickslab/fenwicks/blob/master/tutorials/tutorial3_kaggle_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -qq -U cufflinks

In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import functools

In [3]:
if tf.io.gfile.exists('./fenwicks'):
  tf.io.gfile.rmtree('./fenwicks')
!git clone -q https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

W0619 18:56:53.356706 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/imports.py:19: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0619 18:56:53.358531 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/imports.py:19: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0619 18:56:54.439722 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/optim.py:4: The name tf.train.MomentumOptimizer is deprecated. Please use tf.compat.v1.train.MomentumOptimizer instead.

W0619 18:56:54.441459 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/optim.py:26: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0619 18:56:54.444072 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/train.py:164: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction

In [4]:
BATCH_SIZE = 512 #@param ["512", "256", "128"] {type:"raw"}
MOMENTUM = 0.9 #@param ["0.9", "0.95", "0.975"] {type:"raw"}
WEIGHT_DECAY = 0.000125 #@param ["0.000125", "0.00025", "0.0005"] {type:"raw"}
LEARNING_RATE = 0.4 #@param ["0.4", "0.2", "0.1"] {type:"raw"}
EPOCHS = 24 #@param {type:"slider", min:0, max:100, step:1}
WARMUP = 5 #@param {type:"slider", min:0, max:24, step:1}
ROOT_DIR = 'gs://gs_colab'
PROJECT = 'tutorial3'
VALID_BATCH_SIZE = 10000
PRED_BATCH_SIZE = 10000

In [5]:
fw.colab_utils.setup_gcs()

W0619 18:56:59.211204 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/utils/colab.py:21: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
fw.colab_utils.setup_kaggle_from_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive


In [7]:
!kaggle $fw.datasets.URLs.KAGGLE_CIFAR10

  0% 0.00/3.04M [00:00<?, ?B/s]
100% 3.04M/3.04M [00:00<00:00, 104MB/s]
 99% 601M/610M [00:10<00:00, 96.1MB/s]
100% 610M/610M [00:10<00:00, 58.5MB/s]
 94% 98.0M/105M [00:02<00:00, 30.1MB/s]
100% 105M/105M [00:03<00:00, 33.5MB/s] 
  0% 0.00/575k [00:00<?, ?B/s]
100% 575k/575k [00:00<00:00, 181MB/s]


In [8]:
!apt -qq install libarchive-dev
!apt -qq autoremove
!pip install -qq libarchive

data_dir_local = './data'
fw.io.unzip(['./train.7z', './test.7z'], data_dir_local)

The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  libarchive-dev
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 470 kB of archives.
After this operation, 1,621 kB of additional disk space will be used.
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.3_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.3) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.3) ...
The following packages will be REMOVED:
  libnvidia-common-410
0 upgraded, 0 newly installed, 1 to remove and 8 not upgraded.
After this operation, 33.8 kB disk space will be freed.
(Reading database ... 130968 files and directories currently installed.)
Removing

W0619 18:58:14.147784 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/io.py:157: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0619 18:58:14.149814 140522069198720 io.py:157] Decompressing: /content/train.7z


I0619 18:58:29.291189 140522069198720 io.py:157] Decompressing: /content/test.7z


In [9]:
data_dir_local_train = os.path.join(data_dir_local, 'train')
data_dir_local_test = os.path.join(data_dir_local, 'test')
fw.image.check_rgb(data_dir_local_train, file_ext='png')
fw.image.check_rgb(data_dir_local_test, file_ext='png')

In [10]:
data_dir, work_dir = fw.io.get_project_dirs(ROOT_DIR, PROJECT)

In [11]:
local_train_fn = os.path.join(data_dir_local, "train.tfrec")

paths_train, y_train, labels = fw.data.data_dir_label_csv_tfrecord(
  data_dir=data_dir_local_train, csv_fn='./trainLabels.csv', output_fn=local_train_fn, file_ext='png')

In [12]:
n_train, n_classes = len(paths_train), len(labels)
n_train, n_classes

(50000, 10)

In [13]:
fw.plt.plot_counts_pie(y_train, labels, w=450)

In [14]:
fw.anim.show_image_files(paths_train, h_inch=1, w_inch=1)

In [15]:
X_train_mean, X_train_std, img_size, _ = fw.image.compute_mean_std(local_train_fn, n_train, batch_size=100)

W0619 19:00:46.608996 140522069198720 deprecation.py:323] From /content/fenwicks/data.py:400: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
W0619 19:00:46.642628 140522069198720 deprecation.py:323] From /content/fenwicks/data.py:404: map_and_batch (from tensorflow.python.data.experimental.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
W0619 19:00:46.645510 140522069198720 deprecation_wrapper.py:119] From /content/fenwi

In [16]:
train_fn = os.path.join(data_dir, "train.tfrec")
fw.io.upload_to_gcs(local_train_fn, train_fn)

In [17]:
test_fn = os.path.join(data_dir, "test.tfrec")

path_test = fw.data.data_dir_no_label_tfrecord(data_dir=data_dir_local_test, output_fn=test_fn, file_ext='png')

n_test = len(path_test)
n_test

300000

In [18]:
train_tfms = [
  fw.transform.tfm_standard_scaler(X_train_mean, X_train_std),
  fw.transform.tfm_pad_crop(4),
  fw.transform.tfm_random_flip(),
  fw.transform.tfm_cutout(8, 8),
  fw.transform.tfm_set_shape(img_size, img_size),
]

test_tfms = [
  fw.transform.tfm_standard_scaler(X_train_mean, X_train_std),
  fw.transform.tfm_set_shape(img_size, img_size),
]

parser_train = fw.data.get_tfexample_image_parser(train_tfms)
parser_test = fw.data.get_tfexample_image_parser(test_tfms, has_label=False)

In [19]:
train_input_func = lambda params: fw.data.tfrecord_ds(train_fn, parser_train, batch_size=params['batch_size'], training=True)
test_input_func = lambda params: fw.data.tfrecord_ds(test_fn, parser_test, batch_size=params['batch_size'], training=False)

In [20]:
def build_nn(c=64, weight=0.125):
  model = fw.Sequential()
  model.add(fw.layers.ConvBN(c, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvResBlk(c*2, res_convs=2, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvBlk(c*4, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvResBlk(c*8, res_convs=2, **fw.layers.PYTORCH_PARAMS))
  model.add(tf.keras.layers.GlobalMaxPool2D())
  model.add(fw.layers.Classifier(n_classes, kernel_initializer=fw.layers.init_pytorch, weight=weight))
  return model

In [21]:
steps_per_epoch = n_train // BATCH_SIZE
total_steps = steps_per_epoch * EPOCHS
warmup_steps = steps_per_epoch * WARMUP

In [22]:
lr_decay = fw.train.linear_decay()
lr_func = fw.train.one_cycle_lr(LEARNING_RATE/BATCH_SIZE, total_steps, warmup_steps, lr_decay)
fw.plt.plot_lr_func(lr_func, total_steps)

W0619 19:02:19.265532 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/train.py:52: The name tf.train.polynomial_decay is deprecated. Please use tf.compat.v1.train.polynomial_decay instead.

W0619 19:02:19.268852 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/visualization/plotly.py:35: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0619 19:02:19.288897 140522069198720 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/optimizer_v2/learning_rate_schedule.py:409: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0619 19:02:19.305309 140522069198720 deprecation_wrapper.py:119] From /content/fenwicks/visualization/plotly.py:39: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



In [23]:
opt_func = fw.train.sgd_optimizer(lr_func, mom=MOMENTUM, wd=WEIGHT_DECAY*BATCH_SIZE)
model_func = fw.train.get_clf_model_func(build_nn, opt_func, reduction=tf.losses.Reduction.SUM)

In [24]:
est = fw.train.get_tpu_estimator(steps_per_epoch, model_func, work_dir, trn_bs=BATCH_SIZE, val_bs=VALID_BATCH_SIZE, pred_bs=PRED_BATCH_SIZE)
est.train(train_input_func, steps=total_steps)

W0619 19:02:21.321299 140522069198720 estimator.py:1984] Estimator's model_fn (<function get_clf_model_func.<locals>.model_func at 0x7fcd3330e158>) includes params argument, but params are not passed to Estimator.
I0619 19:02:21.324752 140522069198720 estimator.py:209] Using config: {'_model_dir': 'gs://gs_colab/work/tutorial3/2019-06-19-19:02:21', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.100.50.194:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.

In [25]:
y_preds = []
test_ids = []
for i, pred in enumerate(est.predict(test_input_func)):
  y_preds.append(pred['y_pred'])
  fn = os.path.basename(path_test[i])
  test_ids.append(fn[:-4])

I0619 19:03:45.682924 140522069198720 estimator.py:1145] Calling model_fn.
I0619 19:03:46.547985 140522069198720 estimator.py:1147] Done calling model_fn.
I0619 19:03:46.558412 140522069198720 tpu_estimator.py:499] TPU job name worker
I0619 19:03:46.729499 140522069198720 monitored_session.py:240] Graph was finalized.
W0619 19:03:46.731360 140522069198720 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
I0619 19:03:46.792628 140522069198720 saver.py:1280] Restoring parameters from gs://gs_colab/work/tutorial3/2019-06-19-19:02:21/model.ckpt-2328
I0619 19:03:48.209443 140522069198720 session_manager.py:500] Running local_init_op.
I0619 19:03:48.264492 140522069198720 session_manager.py:502] Done running local_init_op.
I

In [26]:
d = {'id': test_ids, 'label': [labels[y] for y in y_preds]}
df = pd.DataFrame(data=d)
df

,id,label
0,21816,cat
1,147187,truck
2,224226,airplane
3,44885,truck
4,45793,cat
5,21404,truck
6,165075,deer
7,61578,frog
8,220816,truck
9,23811,frog


In [27]:
df.to_csv('submission.csv', index=False)

In [28]:
!kaggle competitions submit -c cifar-10 -f submission.csv -m "Tutorial 3"

100% 3.53M/3.53M [00:01<00:00, 2.51MB/s]
Successfully submitted to CIFAR-10 - Object Recognition in Images

In [31]:
!kaggle competitions submissions -c cifar-10

fileName        date                 description  status    publicScore  privateScore  
--------------  -------------------  -----------  --------  -----------  ------------  
submission.csv  2019-06-19 19:04:18  Tutorial 3   complete  0.93850      0.93850       
submission.csv  2019-05-22 19:21:40  Tutorial 3   complete  0.93960      0.93960       
submission.csv  2019-05-15 06:36:24  Tutorial 3   complete  0.94320      0.94320       
submission.csv  2019-05-15 05:57:41  Tutorial 3   complete  0.09710      0.09710       
submission.csv  2019-05-12 05:46:00  Tutorial 3   complete  0.94200      0.94200       
submission.csv  2019-05-08 15:36:23  Tutorial 3   complete  0.94000      0.94000       
submission.csv  2019-05-08 11:49:42  Tutorial 3   complete  0.94250      0.94250       
submission.csv  2019-05-02 17:01:11  Tutorial 3   complete  0.93970      0.93970       
submission.csv  2019-04-25 18:36:36  Tutorial 3   complete  0.94020      0.94020       
submission.csv  2019-04-21 18:15

In [30]:
fw.io.create_clean_dir(work_dir)